In [1]:
%autosave 60
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Autosaving every 60 seconds


## Auxiliary Functions

We need the parser for first order formulas, hence we import it.

In [2]:
import folParser as fp

def parse(s):
    return fp.LogicParser(s).parse()

The resolution calculus works with clauses.  The module <tt>folCNF</tt> implements the function $\texttt{normalize}(f)$ that turns a formula $f$ into a set of clauses that is equisatisfiable to $f$.

In [3]:
import folCNF as cnf

The module <tt>unify</tt> implements unification and the appplication of a substitution $\sigma$ to a term:
<ol>
<li>The function $\texttt{mgu}(s, t)$, which is defined in the module <tt>unify</tt>, 
    takes two terms $s$ and $t$ and computes the 
    <em style="color:blue;">most general unifier</em> of $s$ and $t$.
</li>
<li>The function $\texttt{apply}(t, \sigma)$ takes a term $t$ and a substitution $\sigma$ and
    computes the <em  style="color:blue;">application</em> $t\sigma$ of $\sigma$ to $t$.
</li>
</ol>

In [4]:
from unify import mgu, apply

The call $\texttt{arb}(S)$ returns an arbitrary element from the set $S$.

In [5]:
def arb(S):
    for x in S:
        return x

Given a literal $l$ the function $\texttt{complement}(l)$ computes the 
<em style="color:blue;">complement</em> $\overline{\,l\,}$ of the literal $l$.

In [6]:
def complement(l):
    "Compute the complement of the literal l."
    if l[0] == '¬':
        return l[1]
    else:
        return ('¬', l)

In [7]:
complement(('p', 'x'))

('¬', ('p', 'x'))

In [8]:
complement(('¬', ('p', 'x')))

('p', 'x')

Given a clause $C$, the function $\texttt{collectVariables}(C)$ computes the set of all variables occurring in $C$.  The function $\texttt{collectVariables}$ can also collect the variables occurring in a literal or a term.

In [9]:
def collectVariables(C):
    if isinstance(C, frozenset):
        return { x for literal in C 
                   for x in collectVariables(literal) 
               }
    if isinstance(C, str): # C is a variable
        return { C }
    if C[0] == '¬':
        return collectVariables(C[1])
    # C must be a term or an atomic formula
    args = C[1:]
    return { x for t in args for x in collectVariables(t) }

In [10]:
s = '∀g:∀c:(Grandparent(g, c) ↔ ∃p: (Parent(g, p) ∧ Parent(p, c)))'
f = parse(s)
f

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [11]:
Clauses = cnf.normalize(f)
Clauses

{frozenset({('Grandparent', 'g', 'c'),
            ('¬', ('Parent', 'a', 'c')),
            ('¬', ('Parent', 'g', 'a'))}),
 frozenset({('Parent', 'g', ('sk1', 'g', 'c')),
            ('¬', ('Grandparent', 'g', 'c'))}),
 frozenset({('Parent', ('sk1', 'g', 'c'), 'c'),
            ('¬', ('Grandparent', 'g', 'c'))})}

In [12]:
for C in Clauses:
    print(f'collectVariables({set(C)}) = \n\t{collectVariables(C)}')

collectVariables({('¬', ('Parent', 'a', 'c')), ('¬', ('Parent', 'g', 'a')), ('Grandparent', 'g', 'c')}) = 
	{'c', 'g', 'a'}
collectVariables({('¬', ('Grandparent', 'g', 'c')), ('Parent', 'g', ('sk1', 'g', 'c'))}) = 
	{'c', 'g'}
collectVariables({('¬', ('Grandparent', 'g', 'c')), ('Parent', ('sk1', 'g', 'c'), 'c')}) = 
	{'c', 'g'}


In [13]:
from string import ascii_lowercase

The function $\texttt{renameVariables}(f, g)$ renames the variables in the formula $f$ so that they are different from the variables occurring in the formula $g$.

In [14]:
def renameVariables(f, g=frozenset()):
    OldVars = collectVariables(f)
    NewVars = set(ascii_lowercase) - collectVariables(g)
    NewVars = sorted(list(NewVars))
    sigma   = { x: NewVars[i] for (i, x) in enumerate(OldVars) }
    return apply(f, sigma)

In [15]:
renameVariables(f)

('∀',
 'b',
 ('∀',
  'a',
  ('↔',
   ('Grandparent', 'b', 'a'),
   ('∃', 'c', ('∧', ('Parent', 'b', 'c'), ('Parent', 'c', 'a'))))))

# A Calculus for First Order Logic

The <em style="color:blue;">resolution</em> rule is an inference rule that is defined as follows: If
<ol>
<li> $C_1$ and $C_2$ are clauses from first order logic,</li>
<li> $p(s_1,\cdots,s_n)$ and $p(t_1,\cdots,t_n)$ are atomic formulas,</li> 
<li> the syntactical equation $p(s_1,\cdots,s_n) \doteq p(t_1,\cdots,t_n)$ is solvable and
     $$ \mu = \mathtt{mgu}\bigl(p(s_1,\cdots,s_n), p(t_1,\cdots,t_n)\bigr), $$</li> 
</ol>
then
$$\frac{C_1 \cup\{ p(s_1,\cdots,s_n)\} \quad\quad \{\neg p(t_1,\cdots,t_n)\} \cup C_2}{
                 C_1\mu \cup C_2\mu} 
$$
is an application of the resolution rule.

Given a two clauses <tt>C1</tt> and <tt>C2</tt>, the function $\texttt{resolve}(\texttt{C1}, \texttt{C2})$ computes a set of all clauses that can be inferred from <tt>C1</tt> and <tt>C2</tt> by applying the resolution rule.

In [16]:
def resolve(C1, C2):
    C2 = renameVariables(C2, C1)
    Result = set()
    for L1 in C1:
        for L2 in C2:
            mu = mgu(L1, complement(L2))
            if mu != None:
                C1C2 = apply((C1 - { L1 }) | (C2 - { L2 }), mu)
                Result.add(C1C2)
    return Result

## Some Formulas for Testing

According to the book 
<a href="https://www.springer.com/la/book/9780817647629">Logic for Computer Science</a> written by 
<a href="https://de.wikipedia.org/wiki/Uwe_Schöning">Uwe Schöning</a>, the theory of red dragons is
given by the following axioms:
<ol>
<li>
Every dragon is happy if all its children can fly:
$$ f_1 := \forall x: \Bigl(\forall y: \big(\texttt{Child}(y,x) \rightarrow \texttt{CanFly}(y)\big) \rightarrow \texttt{Happy}(x)\Bigr) 
$$
</li>
<li> 
All red dragons can fly:
$$
 f_2 := \forall x: \bigl(\texttt{Red}(x) \rightarrow \texttt{CanFly}(x)\bigr)
$$
</li>
<li> The children of red dragons are red themselves:
$$
f_3 := \forall x: \Bigl(\texttt{Red}(x) \rightarrow \forall y:\bigl( \texttt{Child}(y,x) \rightarrow \texttt{Red}(y)\bigr)\Bigr)
$$
</li>
</ol>
We will show that these axioms imply that all red dragons are happy:
$$
 \forall x: \bigl(\texttt{Red}(x) \rightarrow \texttt{Happy}(x)\bigr)
$$
To this end, the formula stating that all red dragons can fly is negated.  
$$
 f_4 := \neg\forall x: \bigl(\texttt{Red}(x) \rightarrow \texttt{Happy}(x)\bigr)
$$
Then we will show that the set consisting of the negated formula together with the axioms is inconsistent:  
$$ \bigl\{ f_1, f_2, f_3, f_4 \} \vdash \bot. $$
We start by defining the formulas.

In [17]:
s1 = '∀x:(∀y:(Child(y, x) → CanFly(y)) → Happy(x))'
s2 = '∀x:(Red(x) → CanFly(x))'
s3 = '∀x:(Red(x) → ∀y:(Child(y, x) → Red(y)))'
s4 = '¬∀x:(Red(x) → Happy(x))'

Next, the formulas are parsed and transformed into clauses.

In [18]:
f1 = parse(s1)
cnf.normalize(f1)

{frozenset({('Child', ('sk2', 'x'), 'x'), ('Happy', 'x')}),
 frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk2', 'x')))})}

In [19]:
f2 = parse(s2)
cnf.normalize(f2)

{frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})}

In [20]:
f3 = parse(s3)
cnf.normalize(f3)

{frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))})}

In [21]:
f4 = parse(s4)
cnf.normalize(f4)

{frozenset({('¬', ('Happy', ('sk3',)))}), frozenset({('Red', ('sk3',))})}

In [22]:
Clauses = cnf.normalize(f1) | cnf.normalize(f2) | cnf.normalize(f3) | cnf.normalize(f4)

We give names to the clauses in order to be able to refer to them.

In [23]:
C1 = frozenset({('Red', ('sk3',))})

In [24]:
C2 = frozenset({('¬', ('Happy', ('sk3',)))})

In [25]:
C3 = frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})

In [26]:
C4 = frozenset({('Child', ('sk2', 'x'), 'x'), ('Happy', 'x')})

In [27]:
C5 = frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk2', 'x')))})

In [28]:
C6 = frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))})

Now we are ready to show that the set containing of these clauses is inconsistent.

In [29]:
C7 = arb(resolve(C1, C6))
C7

frozenset({('Red', 'b'), ('¬', ('Child', 'b', ('sk3',)))})

In [30]:
C8 = arb(resolve(C7, C4))
C8

frozenset({('Happy', ('sk3',)), ('Red', ('sk2', ('sk3',)))})

In [31]:
C9 = arb(resolve(C8, C2))
C9

frozenset({('Red', ('sk2', ('sk3',)))})

In [32]:
C10 = arb(resolve(C9, C3))
C10

frozenset({('CanFly', ('sk2', ('sk3',)))})

In [33]:
C11 = arb(resolve(C10, C5))
C11

frozenset({('Happy', ('sk3',))})

In [34]:
arb(resolve(C11, C2))

frozenset()

As we have derived the empty set, we have shown that all <b style="color:red;">communist</b> dragons are happy!

## Factorization

A calculus which only contains the resolution rule is not complete.  We also need the <em style="color:blue;">factorization</em> rule.  If
<ol>
<li> $C$ is a clause from first order logic,</li>
<li> $p(s_1,\cdots,s_n)$ and $p(t_1,\cdots,t_n)$ are atomic formulas,</li>
<li> the syntactical equation $p(s_1,\cdots,s_n)  \doteq p(t_1,\cdots,t_n)$ is solvable and 
     $$\mu = \mathtt{mgu}\bigl(p(s_1,\cdots,s_n), p(t_1,\cdots,t_n)\bigr), $$</li>
</ol>
then both 
$$ \frac{C \cup \bigl\{p(s_1,\cdots,s_n),\, p(t_1,\cdots,t_n)\bigl\}}{C\mu \cup \bigl\{p(s_1,\cdots,s_n)\mu\bigr\} } $$ 
and
$$ \frac{C \cup \bigl\{ \neg p(s_1,\cdots,s_n),\, \neg p(t_1,\cdots,t_n)\bigl\}}{C\mu \cup \bigl\{\neg p(s_1,\cdots,s_n)\mu\bigr\} }
$$ 
are applications of the factorization rule.

The function $\texttt{factorize}(C)$ takes a clause $C$ from first order logic and computes all clauses that can be derived from $C$ via factorization.

In [35]:
def factorize(C):
    Result = set()
    for L1 in C:
        for L2 in C:
            if L1 != L2:
                mu  = mgu(L1, L2)
                if mu != None:
                    Cmu = apply(C, mu)
                    Result.add(Cmu)
    return Result

The clauses 
<ol>
<li>$C_1 := \forall x: \forall y: P(F(x),y) \vee \forall u: \forall v:P(u, G(v))\qquad\qquad$ and</li>
<li>$C_2 := \forall x: \forall y: \bigl(\neg P(F(x),y)\bigr) \vee \forall u: \forall v: \bigl(\neg P(u, G(v))\bigr)$</li>
</ol>
are inconsistent.  However, the resolution rule alone is not sufficient to show that the set 
$\{C_1, C_2\}$ is inconsistent.

In [36]:
C1 = arb(cnf.normalize(parse('∀x:∀y:P(F(x),y) ∨ ∀u:∀v:P(u,G(v))')))
C1

frozenset({('P', 'u', ('G', 'v')), ('P', ('F', 'x'), 'y')})

In [37]:
C2 = arb(cnf.normalize(parse('∀x:∀y:(¬P(F(x),y)) ∨ ∀u:∀v:(¬P(u,G(v)))')))
C2

frozenset({('¬', ('P', 'u', ('G', 'v'))), ('¬', ('P', ('F', 'x'), 'y'))})

In [38]:
C3 = arb(factorize(C1))
C3

frozenset({('P', ('F', 'x'), ('G', 'v'))})

In [39]:
C4 = arb(factorize(C2))
C4

frozenset({('¬', ('P', ('F', 'x'), ('G', 'v')))})

In [40]:
arb(resolve(C3, C4))

frozenset()

## Automatic Theorem Proving

Given a set of clauses <tt>Clauses</tt>, the function
$\texttt{infere}(\texttt{Cs1}, \texttt{Cs2})$ returns all possible clauses that result from:
<ol>
    <li>The resolution of two clauses $C_1, C_2 \in \texttt{Clauses}$, or</li>
    <li>the factorization of a clause $C \in \texttt{Clauses}$.</li>
</ol>

In [46]:
def infere(Clauses):
    Result =  { C for C1 in Clauses
                  for C2 in Clauses
                  for C  in resolve(C1, C2)
                  if len(C) <= 3            # dirty hack to speed up the prover
              }
    Result |= { C for C1 in Clauses for C in factorize(C1) }
    return Result 

In [47]:
def prettyPrint(Clauses):
    for C in Clauses:
        print(set(C))

In [48]:
prettyPrint(Clauses)

{('CanFly', 'x'), ('¬', ('Red', 'x'))}
{('Red', ('sk5',))}
{('¬', ('Child', 'y', 'x')), ('Red', 'y'), ('¬', ('Red', 'x'))}
{('Happy', 'x'), ('¬', ('CanFly', ('sk4', 'x')))}
{('¬', ('Happy', ('sk5',)))}
{('Happy', 'x'), ('Child', ('sk4', 'x'), 'x')}


The function $\texttt{saturate}(\texttt{Cs})$ takes a set of clauses $\texttt{Cs}$ as input and tries to infere the empty clause.  If it is not possible to infer the empty clause, the function runs until memory is exhausted.

In [49]:
def saturate(Clauses):
    Cs = Clauses.copy()
    cnt = 1
    while frozenset() not in Cs:
        NewCs = infere(Cs)
        print(f'cnt = {cnt}, number of inferred clauses: {len(NewCs)}')
        Cs |= NewCs
        cnt += 1
    return True

In [50]:
%time saturate(Clauses)

cnt = 1, number of inferred clauses: 11
cnt = 2, number of inferred clauses: 57
cnt = 3, number of inferred clauses: 346
CPU times: user 145 ms, sys: 1.71 ms, total: 147 ms
Wall time: 145 ms


True